In [2]:
!pip3 install pyspark


     |████████████████████████████████| 212.4MB 66kB/s 
     |████████████████████████████████| 204kB 19.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=b07cc569d4f93cf635fdc96f53ee044bcc0afcff39e0bc6ef724097543e30815
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [3]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['spark-3.1.2']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$spark_version/$spark_version-bin-hadoop2.7.tgz
!tar xf $spark_version-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 

In [4]:
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()

In [5]:
#create dataframe
#already has a list of words in df. skipping the tokenization step for now. that step usually takes input and seperates it into a list of words. by creating df that already contains list of words, we are replicating this step
sentenceData = spark.createDataFrame([
    (0, ["Big", "data", "is", "super", "powerful"]),
    (1, ["This", "is", "going", "to", "be", "epic"]),
], ["id", "raw"])                              
      
sentenceData.show(truncate=False)

+---+--------------------------------+
|id |raw                             |
+---+--------------------------------+
|0  |[Big, data, is, super, powerful]|
|1  |[This, is, going, to, be, epic] |
+---+--------------------------------+



In [6]:
# Import stop words library
from pyspark.ml.feature import StopWordsRemover

In [7]:
# Run the Remover
#takes the input column that will be passed into the function and an output column to add the results
remover = StopWordsRemover(inputCol="raw", outputCol="filtered")

In [8]:
#transform and show data
remover.transform(sentenceData).show(truncate=False)

+---+--------------------------------+----------------------------+
|id |raw                             |filtered                    |
+---+--------------------------------+----------------------------+
|0  |[Big, data, is, super, powerful]|[Big, data, super, powerful]|
|1  |[This, is, going, to, be, epic] |[going, epic]               |
+---+--------------------------------+----------------------------+



In [9]:
#skill drill. Combine both tokenizer and StopWordsRemover on a DataFrame that isn't already broken out into a list of words.
skilldrillData = spark.createDataFrame([
    (0, ["Today", "is", "a", "good", "day"]),
    (1, ["Make", "tomorrow", "even", "better"]),
], ["id", "raw"])                              
      
skilldrillData.show(truncate=False)

+---+------------------------------+
|id |raw                           |
+---+------------------------------+
|0  |[Today, is, a, good, day]     |
|1  |[Make, tomorrow, even, better]|
+---+------------------------------+



In [10]:
remover = StopWordsRemover(inputCol="raw", outputCol="filtered")

In [12]:
remover.transform(skilldrillData).show(truncate=False)

+---+------------------------------+------------------------------+
|id |raw                           |filtered                      |
+---+------------------------------+------------------------------+
|0  |[Today, is, a, good, day]     |[Today, good, day]            |
|1  |[Make, tomorrow, even, better]|[Make, tomorrow, even, better]|
+---+------------------------------+------------------------------+

